# Loops & Orbits &mdash; Week 3 &mdash; Day 1 &mdash; Jupyter Notebook

## Newton's Cannon with `numpy`

First, define all of the constants and functions that are needed in the rest of the code.

In [22]:
# Important math functions and the constant pi:
from math import sin, cos, atan2, sqrt, pi

import numpy as np

# The following import statement makes the plotting library available to us. There is also a mysterious
# statement to work around a known Jupyter bug: https://github.com/jupyter/notebook/issues/3670
%matplotlib inline
import matplotlib.pyplot as plt

import unittest

In [23]:
# The following two parameters are get us the strength of gravity at various heights:
g = 9.81                    # 9.81 m/s**2 at Earth surface
radius_of_earth = 6371000.0 # Earth radius in meters

# The following two variables define the cannonball launch conditions:
initial_position = np.array([0.0, radius_of_earth + 500000.0])  # cannon is on a 500 km high mountain
initial_velocity = np.array([8000.0, 0.0])                      # a good range of speeds to try is 3000 to 8000 m/s

# The following two parameters establish the simulation time step and its maximum total duration:
delta_t = 30.0              # simulation time step in seconds
simulation_time_steps = 80  # 40 minutes worth for a delta_t of 30.0

## Functions for Working with Vectors

All of these have already appeared in previous notebooks, but they must be re-implemented to take and return `np.ndarray`.

In [50]:
# computes horizontal and vertical components of a vector and returns them as a tuple
def vector_from_length_and_angle(length: float, angle: float) -> np.ndarray:
    # we are working in degrees -- python's are expecting radians -- convert degrees to radian
    angle_in_radians = angle * pi / 180.0
    x_component = length * cos(angle_in_radians)
    y_component = length * sin(angle_in_radians)
    return np.array([x_component, y_component])

# get angle from components using atan2 version of arctangent function -- result is converted to degrees
def angle_from_vector(vector: np.ndarray) -> float:
    # use the arctangent function
    angle_in_radians = atan2(vector[1], vector[0])  
    # we are working in degrees -- python's functions return radians -- convert radians to degrees
    angle = angle_in_radians * 180.0 / pi
    # return the result
    return angle

# get length from components using Pythagorean theorem
def length_from_vector(vector: np.ndarray) -> float:
    length_squared = np.sum(vector**2)
    return sqrt(length_squared)

### Unit tests ###

class VectorTests(unittest.TestCase):
    
    def test_vector_from_length_and_angle(self):
        vector = vector_from_length_and_angle(1000.0, 30.0)
        y_component = vector[1]
        self.assertAlmostEqual(y_component, 500.0)
        
    def test_angle_from_vector(self):
        angle = angle_from_vector(np.array([866.02540378, 500.0]))
        self.assertAlmostEqual(angle, 30.0)
        
    def test_length_from_vector(self):
        length = length_from_vector(np.array([5.0, 12.0]))
        self.assertAlmostEqual(length, 13.0)

testSuite = unittest.TestLoader().loadTestsFromName("__main__.VectorTests")
testRunner = unittest.TextTestRunner(verbosity=2)
testRunner.run(testSuite)

test_angle_from_vector (__main__.VectorTests) ... ok
test_length_from_vector (__main__.VectorTests) ... ok
test_vector_from_length_and_angle (__main__.VectorTests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

## Functions to Get Acceleration of Gravity

**Below is the suite of four functions for you to implement.** Each function has comments saying what it should do.

*Make your life easy!* Use the functions for working with vectors above. If you find you are using sqrt, cos, sin, etc. below you are re-doing work that is already complete and correct in the functions above.

*Each function has two unit tests, most of which are failing.* Keep working on your implementations until the 
unit tests pass. Then go on to the rest of the notebook.

In [2]:
import numpy as np

In [67]:
def impacted(position: np.ndarray) -> bool:
    # this function should return False if the cannonball is above the Earth's surface
    # it should return True if the cannonball is below the Earth's surface -- this will end the while loop
    radius = length_from_vector(position)
    return radius <= radius_of_earth

def strength_of_gravity(position: np.ndarray) -> float:
    # this function encodes the strength of gravity as a function of distance from the center of the Earth
    # it should use Newton's 1/r**2 formula (see whiteboard)
    radius = length_from_vector(position)
    strength = g * radius_of_earth**2 / radius**2
    return strength

def direction_of_gravity(position: np.ndarray) -> np.ndarray:
    # this function encodes the direction of gravity (the angle)
    # gravity is attractive -- it always points toward the center of the Earth
    direction = angle_from_vector(position) + 180.0
    return direction

def acceleration_of_gravity(position: np.ndarray) -> np.ndarray:
    # using the strength and direction functions you have just implemented compute and
    # returns a 2x1 array for the acceleration of gravity
    strength = strength_of_gravity(position)
    direction = direction_of_gravity(position)
    acceleration = vector_from_length_and_angle(strength, direction)
    return acceleration

### Unit tests ###

class GravityTests(unittest.TestCase):

    # tests of impacted
    
    def test_has_impacted(self):
        position = np.array([1000.0, 2000.0])
        has_impacted = impacted(position)
        self.assertTrue(has_impacted)
        
    def test_not_impacted(self):
        position = np.array([-10000000.0, -2000.0])
        not_impacted = impacted(position)
        self.assertFalse(not_impacted)
        
    # tests of strength

    def test_strength_of_gravity(self):
        position = np.array([radius_of_earth, 0.0])
        strength = strength_of_gravity(position)
        self.assertAlmostEqual(strength, g)

    def test_strength_of_gravity_high_up(self):
        strength = strength_of_gravity(0, 2.0 * radius_of_earth)
        self.assertAlmostEqual(strength, 0.25 * g)
        
    # tests of direction
        
    def test_direction_of_gravity_left(self):
        direction = direction_of_gravity(100.0, 0.0)
        self.assertEqual(direction, 180.0)

    def test_direction_of_gravity_up(self):
        direction = direction_of_gravity(0.0, -100.0)
        self.assertEqual(direction, 90.0)
        
    # tests of acceleration

    def test_acceleration_of_gravity_g_left(self):
        acceleration = acceleration_of_gravity(np.array([radius_of_earth, 0.0]))
        self.assertAlmostEqual(acceleration[0], -g)
        self.assertAlmostEqual(acceleration[1], 0.0)

    def test_acceleration_of_gravity_4g_up(self):
        acceleration = acceleration_of_gravity(np.array([0.0, -radius_of_earth / 2.0]))
        self.assertAlmostEqual(acceleration[0], 0.0)
        self.assertAlmostEqual(acceleration[1], 4.0 * g)

testSuite = unittest.TestLoader().loadTestsFromName("__main__.GravityTests")
testRunner = unittest.TextTestRunner(verbosity=2)
testRunner.run(testSuite)


test_acceleration_of_gravity_4g_up (__main__.GravityTests) ... ok
test_acceleration_of_gravity_g_left (__main__.GravityTests) ... ok
test_direction_of_gravity_left (__main__.GravityTests) ... ERROR
test_direction_of_gravity_up (__main__.GravityTests) ... ERROR
test_has_impacted (__main__.GravityTests) ... ok
test_not_impacted (__main__.GravityTests) ... ok
test_strength_of_gravity (__main__.GravityTests) ... ok
test_strength_of_gravity_high_up (__main__.GravityTests) ... ERROR

ERROR: test_direction_of_gravity_left (__main__.GravityTests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-67-353fde4335b7>", line 58, in test_direction_of_gravity_left
    direction = direction_of_gravity(100.0, 0.0)
TypeError: direction_of_gravity() takes 1 positional argument but 2 were given

ERROR: test_direction_of_gravity_up (__main__.GravityTests)
----------------------------------------------------------------------
Trac

<unittest.runner.TextTestResult run=8 errors=3 failures=0>

## The While Loop That Does the Work

**There is nothing for you to change below. You can just execute it once you've got your functions implemented.**

In [ ]:
# Initialize the x and y velocities
x_velocities = [initial_speed]
y_velocities = [0.0]
# Initialize the x and y positions
x_positions = [0.0]
y_positions = [radius_of_earth + initial_height_above_earth]
# Initialize the times
times = [0.0]

# We want to go until the ball is over the fence. It seems our code often runs
# without terminating. Add an extra test to stop it from going more than 100 times.
# This is the first time you have seen the logical and operator!
while not impacted(x_positions[-1], y_positions[-1]) and len(times) * delta_t <= maximum_duration:
    #
    # get all the before values
    #
    # velocities
    before_x_velocity = x_velocities[-1]
    before_y_velocity = y_velocities[-1]
    # positions
    before_x_position = x_positions[-1]
    before_y_position = y_positions[-1]
    # time
    before_time = times[-1]
    #
    # use the new acceleration_with_drag function to get the accelerations
    #
    x_acceleration, y_acceleration = acceleration_of_gravity(before_x_position, before_y_position)
    #
    # Euler-Cromer update code
    #
    # update the x and y velocities
    after_x_velocity = before_x_velocity + delta_t * x_acceleration
    after_y_velocity = before_y_velocity + delta_t * y_acceleration
    # update the x and y positions
    after_x_position = before_x_position + delta_t * after_x_velocity
    after_y_position = before_y_position + delta_t * after_y_velocity
    # update time
    after_time = before_time + delta_t
    #
    # append all the after values to their lists
    #
    x_velocities.append(after_x_velocity)
    y_velocities.append(after_y_velocity)
    x_positions.append(after_x_position)
    y_positions.append(after_y_position)
    times.append(after_time)


## Graph

**Execute this to make a graph that looks a little like [the diagram Newton made](https://upload.wikimedia.org/wikipedia/commons/thumb/2/2f/Newton%27s_Principia_%281846%29.djvu/page519-1024px-Newton%27s_Principia_%281846%29.djvu.jpg).**

In [ ]:

plt.figure(figsize=(8, 8))

plt.scatter(x_positions, y_positions)

plt.xlabel("x position (m)")
plt.ylabel("y position (m)")

# Some gibberish that draws a big blue circle representing the Earth:
earth = plt.Circle((0, 0), radius_of_earth, color='b')
plt.gcf().gca().add_artist(earth)

# Make the plot big enough to show the entire Earth:
plot_limit = 8000000
plt.xlim(-plot_limit, plot_limit)
plt.ylim(-plot_limit, plot_limit)

plt.show()

## Playing with the Simulation and Checkout Questions

Look at arc VF in [the diagram Newton made](https://upload.wikimedia.org/wikipedia/commons/thumb/2/2f/Newton%27s_Principia_%281846%29.djvu/page519-1024px-Newton%27s_Principia_%281846%29.djvu.jpg). For that arc the cannonball goes about 20% of the way around the Earth.

Play with `initial_speed` in the initialization cell. Each time you change it, re-execute the entire notebook.

A good range of values to try is 3000 to 8000 m/s. What makes an arc that is most like Newton's arc VF?

Show me or Ben that you can set a breakpoint in your `strength_of_gravity` function and inspect what is happening.